## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "i like a dog",
    "books are expensive"    
]
data_size = len(corpus)

# 0: noun/pronoun - 1: verb - others - 2
labels = [[0, 1, 2, 0],
          [0, 1, 2]]

# Define the max vocabulary size and sequence length
vocab_size = 9
sequence_length = 4

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'dog': 5,
 'books': 4,
 '<unk>': 0,
 'expensive': 6,
 '<pad>': 1,
 'i': 7,
 'a': 2,
 'are': 3,
 'like': 8}

In [3]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length, sequence_label):
    tokens = tokenizer(text)
    
    token_ids = [vocab[token] for token in tokens]    
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    sequence_label = sequence_label + [3] * (sequence_length - len(tokens))
    
    return torch.tensor(token_ids, dtype=torch.long), torch.tensor(sequence_label, dtype=torch.long)

# Vectorize the samples
sentence_vecs = []
label_vecs = []
for sentence, labels in zip(corpus, labels):
    sentence_vec, labels_vec = vectorize(sentence, vocab, sequence_length, labels)
    sentence_vecs.append(sentence_vec)
    label_vecs.append(labels_vec)

In [4]:
for v in sentence_vecs:
    print(v)

tensor([7, 8, 2, 5])
tensor([4, 3, 6, 1])


In [5]:
for v in label_vecs:
    print(v)

tensor([0, 1, 2, 0])
tensor([0, 1, 2, 3])


## Model

In [6]:
class POS_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.fc = nn.Linear(4, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x.permute(0, 2, 1)

model = POS_Model(vocab_size, 4)

In [7]:
input_1 = torch.tensor([[7, 8, 2, 5]], dtype=torch.long)
output = model(input_1)
print(output.shape)

torch.Size([1, 4, 4])


## Train with full data

In [8]:
criterion = nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
input_data = torch.tensor( [[7, 8, 2, 5],
                            [4, 3, 6, 1]], dtype=torch.long)
label_data = torch.tensor([[0, 1, 2, 0],
                           [0, 1, 2, 3]], dtype=torch.long)

for _ in range(30):
    optimizer.zero_grad()
    outputs = model(input_data)
    loss = criterion(outputs, label_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

1.2048548460006714
1.1662423610687256
1.1281182765960693
1.0905170440673828
1.0534719228744507
1.0170141458511353
0.9811722636222839
0.9459725618362427
0.9114381074905396
0.8775883316993713
0.8444395065307617
0.8120041489601135
0.7802924513816833
0.7493127584457397
0.7190729975700378
0.6895806193351746
0.6608439087867737
0.6328710913658142
0.6056708693504333
0.579251229763031
0.553619921207428
0.5287832617759705
0.5047465562820435
0.48151320219039917
0.45908501744270325
0.43746182322502136
0.416641503572464
0.39662009477615356
0.37739160656929016
0.3589479923248291


In [10]:
outputs = model(input_data)
o_softmax = torch.softmax(outputs, axis=1)

print(o_softmax[0, :, 0])
print(o_softmax[0, :, 1])
print(o_softmax[0, :, 2])
print(o_softmax[0, :, 3])

print()
print(o_softmax[1, :, 0])
print(o_softmax[1, :, 1])
print(o_softmax[1, :, 2])
print(o_softmax[1, :, 3])

tensor([0.5956, 0.1046, 0.1071, 0.1927], grad_fn=<SelectBackward0>)
tensor([0.0680, 0.9081, 0.0144, 0.0095], grad_fn=<SelectBackward0>)
tensor([0.1700, 0.0453, 0.4837, 0.3011], grad_fn=<SelectBackward0>)
tensor([0.7317, 0.1478, 0.0799, 0.0406], grad_fn=<SelectBackward0>)

tensor([0.8704, 0.0190, 0.0543, 0.0564], grad_fn=<SelectBackward0>)
tensor([0.0422, 0.8334, 0.0468, 0.0775], grad_fn=<SelectBackward0>)
tensor([0.1313, 0.1405, 0.6606, 0.0675], grad_fn=<SelectBackward0>)
tensor([0.7610, 0.1279, 0.0286, 0.0825], grad_fn=<SelectBackward0>)


In [11]:
#[[0, 1, 2, 0],
# [0, 1, 2, *]]

o_softmax.argmax(axis=1)

tensor([[0, 1, 2, 0],
        [0, 1, 2, 0]])